In [1]:
import sys
sys.path.append("../src/baselines/TCRP/code/")

In [2]:
import time
import argparse
import numpy as np
import random
import torch
import torch.nn.functional as F
import torch.optim as optim
import os
import glob
from torch.autograd import Variable
import sys
import torch.nn as nn
import pickle
import copy
from data_loading import *
from tcrp_utils import *
from score import *
from inner_loop import InnerLoop
from mlp import mlp
from meta_learner_cv import *

In [3]:

import numpy as np
import pandas as pd

import datetime
import logging
import os
import time
import matplotlib.pyplot as plt

from torch import nn
from torch.nn import functional as F

from functools import cached_property
from sklearn.metrics import average_precision_score, ndcg_score, roc_auc_score

In [4]:
from sklearn.metrics import pairwise_distances


In [5]:
sample_id = 2

In [6]:
cell_line_info = pd.read_csv(
    "/data/ajayago/druid/intermediate/cs6220/raw/sample_info_ccle.csv"
)
cell_line_info.rename(columns={"DepMap_ID": "depmap_id"}, inplace=True)
cell_line_info.set_index("depmap_id", inplace=True)
cell_line_info = pd.DataFrame(cell_line_info.primary_disease)
print(cell_line_info.primary_disease.unique().shape)
cell_line_info

(44,)


,primary_disease
depmap_id,
ACH-000001,Ovarian Cancer
ACH-000002,Leukemia
ACH-000003,Colon/Colorectal Cancer
ACH-000004,Leukemia
ACH-000005,Leukemia
...,...
ACH-002989,Prostate cancer
ACH-002990,Prostate cancer
ACH-002991,Prostate cancer


In [7]:
K = 1
num_trials = 50
meta_batch_size = 10
inner_batch_size = 10
num_updates = 10
num_inner_updates = 1

random.seed(19)
np.random.seed(19)
torch.manual_seed(19)

layer, hidden, meta_lr, inner_lr, tissue_num = (
    1,
    20,
    0.001,
    0.001,
    12,
)  # args.layer, args.hidden, args.meta_lr, args.inner_lr, args.tissue_num

MODEL_STATE_PATH = f"../src/baselines/TCRP/model_checkpoints/TCRP_with_tcga_sample{sample_id}/"

In [8]:
def train_and_persist_model(drug_name):
#     print(drug_name)
    train_cell_line_dataset = pd.read_csv(f"../data/diffusion_pretraining/cl_diffusion_train_sample{sample_id}.csv", index_col = 0)
    train_label_df = pd.read_csv("/data/ajayago/copied_from_cdal1/yiming_data_folder/dataset/CellLine/patient_auc.csv")
    train_label_df[
        train_label_df.drug_name == drug_name
    ].copy()
    train_label_df.set_index(["depmap_id"], inplace=True)
    print(train_label_df.shape)

    train_feature_df = train_cell_line_dataset
    # train_feature_df = train_feature_df#.loc[train_label_df.index.get_level_values(0)]
    filtered_cell_line_info = cell_line_info.loc[
        train_feature_df.index.get_level_values(0)
    ]
    print(filtered_cell_line_info.shape)

    ordered_train_feature_df = []
    for disease_type in filtered_cell_line_info.primary_disease.unique():
        filtered_df = filtered_cell_line_info[
            filtered_cell_line_info.primary_disease == disease_type
        ]
        curr_disease_depmap_ids = filtered_df.index.get_level_values(0)
        ordered_train_feature_df.append(train_feature_df.loc[curr_disease_depmap_ids])

    ordered_train_feature_df = pd.concat(ordered_train_feature_df)
    print(ordered_train_feature_df.shape)

    tissue_index_list = []
    curr_count = 0
    for (
        disease_type,
        sample_count,
    ) in filtered_cell_line_info.primary_disease.value_counts().items():
        tissue_index_list.append(list(range(curr_count, curr_count + sample_count)))
        curr_count += sample_count
    print(curr_count)
    ordered_train_label_df = train_label_df[
        train_label_df.index.isin(ordered_train_feature_df.index.get_level_values(0))
    ].copy()
    print(ordered_train_label_df.shape)

    test_cell_line_dataset = pd.read_csv(f"../data/diffusion_pretraining/cl_diffusion_test_sample{sample_id}.csv", index_col = 0)
    test_label_df = pd.read_csv("/data/ajayago/copied_from_cdal1/yiming_data_folder/dataset/CellLine/patient_auc.csv")
    test_label_df = test_label_df[
        test_label_df.drug_name == drug_name
    ].copy()
    test_label_df.set_index(["depmap_id"], inplace=True)
    print(test_label_df.shape)

    test_feature_df = test_cell_line_dataset#.loc[
    #     test_label_df.index.get_level_values(0)
    # ].copy()
    print(test_feature_df.shape)

    train_dataset = dataset(
        ordered_train_feature_df.to_numpy(),
        (1 - ordered_train_label_df.auc).values.reshape(-1, 1),
    )
    test_dataset = dataset(
        test_feature_df.to_numpy(), (1 - test_label_df.auc).values.reshape(-1, 1)
    )
   
    meta_dataset = train_dataset
    test_dataset = test_dataset
    print(len(ordered_train_feature_df), len(test_feature_df))
    (
        best_train_loss_test_corr_list,
        best_train_corr_test_corr_list,
        best_train_corr_test_scorr_list,
        best_train_scorr_test_scorr_list,
    ) = ([], [], [], [])

    for i in range(num_trials):
        meta_learner = MetaLearner(
            meta_dataset,
            test_dataset,
            K,
            meta_lr,
            inner_lr,
            layer,
            hidden,
            tissue_num,
            meta_batch_size,
            inner_batch_size,
            num_updates,
            num_inner_updates,
            tissue_index_list,
            num_trials,
        )

        (
            best_train_loss_test_corr,
            best_train_corr_test_corr,
            best_train_corr_test_scorr,
            best_train_scorr_test_scorr,
            best_model,
        ) = meta_learner.train()
        best_train_loss_test_corr_list.append(best_train_loss_test_corr)
        best_train_corr_test_corr_list.append(best_train_corr_test_corr)
        best_train_corr_test_scorr_list.append(best_train_corr_test_scorr)
        best_train_scorr_test_scorr_list.append(best_train_scorr_test_scorr)

        # Please uncomment this line to save your pre-train models
        # torch.save(best_model, model_dic + '/model_'+str(K)+'_trail_' + str(i))

    a = np.asarray(best_train_loss_test_corr_list).mean()
    b = np.asarray(best_train_corr_test_corr_list).mean()
    c = np.asarray(best_train_corr_test_scorr_list).mean()
    d = np.asarray(best_train_scorr_test_scorr_list).mean()

    print(
        "best_train_loss_test_corr:",
        float("%.3f" % a),
        "best_train_corr_test_corr",
        float("%.3f" % b),
        "best_train_corr_test_scorr",
        float("%.3f" % c),
        "best_train_scorr_test_scorr",
        float("%.3f" % d),
    )

    torch.save(
        best_model.state_dict(), f"{MODEL_STATE_PATH}/{drug_name.replace('/', '_')}.pth"
    )

In [9]:
for drug_name in [
    "CISPLATIN",
    "PACLITAXEL",
    "5-FLUOROURACIL",
    "CYCLOPHOSPHAMIDE",
    "DOCETAXEL",
    "GEMCITABINE",
    "DOXORUBICIN",
]:
    train_and_persist_model(drug_name)

(226796, 2)
(1569, 1)
(1569, 7776)
1569
(205220, 2)
(539, 2)
(175, 7776)
1569 175
Few shot 0 train_loss: 0.561 train_pearson: -1.0 train_spearman: nan
test_loss: 0.465 test_pearson: 0.071 test_spearman: 0.1
Few shot 1 train_loss: 0.493 train_pearson: -1.0 train_spearman: nan
test_loss: 0.414 test_pearson: -0.003 test_spearman: 0.018
Few shot 2 train_loss: 0.428 train_pearson: -1.0 train_spearman: nan
test_loss: 0.367 test_pearson: -0.029 test_spearman: -0.003
Few shot 3 train_loss: 0.365 train_pearson: -1.0 train_spearman: nan
test_loss: 0.323 test_pearson: -0.04 test_spearman: -0.005
Few shot 4 train_loss: 0.306 train_pearson: -1.0 train_spearman: nan
test_loss: 0.284 test_pearson: -0.047 test_spearman: -0.003
Few shot 5 train_loss: 0.252 train_pearson: -1.0 train_spearman: nan
test_loss: 0.25 test_pearson: -0.052 test_spearman: -0.003
Few shot 6 train_loss: 0.201 train_pearson: -1.0 train_spearman: nan
test_loss: 0.222 test_pearson: -0.056 test_spearman: -0.004
Few shot 7 train_loss:

Finetuning with TCGA

In [10]:
tcga_train_df = pd.read_csv(f"../data/diffusion_pretraining/tcga_diffusion_train_sample{sample_id}.csv", index_col=0)
tcga_train_df.shape

(476, 7776)

In [11]:
patient_responses_df = pd.read_csv("/data/ajayago/copied_from_cdal1/ajayago_home_folder/processed/TCGA_drug_response_010222.csv")
patient_responses_df

,patient.arr,drug.name,response,response_cat,drug
0,TCGA-G2-A2EC,Methotrexate,Partial Response,1,METHOTREXATE
1,TCGA-G2-A2EC,Doxorubicin,Partial Response,1,DOXORUBICIN
2,TCGA-G2-A2EC,Vinblastine,Partial Response,1,VINBLASTINE
3,TCGA-G2-A2EC,Cisplatin,Partial Response,1,CISPLATIN
4,TCGA-G2-A2EJ,Paclitaxel,Stable Disease,0,PACLITAXEL
...,...,...,...,...,...
1244,TCGA-BG-A0VZ,Cisplatin,Complete Response,1,CISPLATIN
1245,TCGA-BG-A0VZ,Paclitaxel,Complete Response,1,PACLITAXEL
1246,TCGA-BG-A0VZ,Doxorubicin,Complete Response,1,DOXORUBICIN
1247,TCGA-BG-A0VT,Carboplatin,Complete Response,1,CARBOPLATIN


In [12]:
train_tcga_with_response = patient_responses_df[patient_responses_df["patient.arr"].isin(tcga_train_df.index)].reset_index(drop=True)
train_tcga_with_response

,patient.arr,drug.name,response,response_cat,drug
0,TCGA-G2-A2EC,Methotrexate,Partial Response,1,METHOTREXATE
1,TCGA-G2-A2EC,Doxorubicin,Partial Response,1,DOXORUBICIN
2,TCGA-G2-A2EC,Vinblastine,Partial Response,1,VINBLASTINE
3,TCGA-G2-A2EC,Cisplatin,Partial Response,1,CISPLATIN
4,TCGA-G2-A2EJ,Paclitaxel,Stable Disease,0,PACLITAXEL
...,...,...,...,...,...
968,TCGA-E6-A8L9,Carboplatin,Complete Response,1,CARBOPLATIN
969,TCGA-2E-A9G8,Paclitaxel,Complete Response,1,PACLITAXEL
970,TCGA-2E-A9G8,Carboplatin,Complete Response,1,CARBOPLATIN
971,TCGA-BG-A0VT,Carboplatin,Complete Response,1,CARBOPLATIN


In [13]:
learning_rate = 0.001

# for drug_name in ["CISPLATIN", "PACLITAXEL", "GEMCITABINE", "DOXORUBICIN", "OLAPARIB"]:
for drug_name in [
    "CISPLATIN",
    "PACLITAXEL",
    "5-FLUOROURACIL",
    "CYCLOPHOSPHAMIDE",
    "DOCETAXEL",
    "GEMCITABINE",
    "DOXORUBICIN",
]:

    if os.path.exists(f"{MODEL_STATE_PATH}/{drug_name.replace('/', '_')}.pth"):
        curr_model = mlp(7776, 1, 20)
        curr_model.load_state_dict(
            torch.load(
                f"{MODEL_STATE_PATH}/{drug_name.replace('/', '_')}.pth",
                map_location=torch.device("cpu"),
            )
        )
        filtered_df = train_tcga_with_response[
            train_tcga_with_response.drug == drug_name
        ].copy()
        patients_with_pos_response = list(
            filtered_df[filtered_df.response_cat == 1]["patient.arr"].values[:2]
        )
        patients_with_neg_response = list(
            filtered_df[filtered_df.response_cat == 0]["patient.arr"].values[:2]
        )
        if (len(patients_with_pos_response) == 0) or ((len(patients_with_neg_response) == 0)):
            print(f"Skipping training for {drug_name} - {patients_with_pos_response}{patients_with_neg_response}")
            continue
        
        fs_criterion = torch.nn.BCEWithLogitsLoss()
        fs_optim = torch.optim.Adam(curr_model.parameters(), lr=learning_rate)
        
        required_patients = patients_with_neg_response + patients_with_pos_response
        few_shot_df = filtered_df[filtered_df["patient.arr"].isin(required_patients)]
        train_tcga_x = torch.tensor(
            tcga_train_df.loc[few_shot_df["patient.arr"].values].to_numpy(),
            device="cpu",
            dtype=torch.float,
        )
        train_tcga_y = torch.tensor(
            few_shot_df.response_cat.to_numpy(), device="cpu", dtype=torch.float
        )

        pred_train_tcga_y, _ = curr_model(train_tcga_x)
        fs_loss = fs_criterion(pred_train_tcga_y.flatten(), train_tcga_y)
        fs_loss.backward()
        fs_optim.step()
        
        torch.save(
            curr_model.state_dict(), f"{MODEL_STATE_PATH}/{drug_name.replace('/', '_')}.pth"
        )

    else:
        print(f"Model for {drug_name} not found")
    break

In [14]:
tcga_test_df = pd.read_csv(f"../data/diffusion_pretraining/tcga_diffusion_test_sample{sample_id}.csv", index_col=0)
tcga_test_df.shape

(120, 7776)

In [15]:
test_tcga_with_response = patient_responses_df[patient_responses_df["patient.arr"].isin(tcga_test_df.index)].reset_index(drop=True)
test_tcga_with_response

,patient.arr,drug.name,response,response_cat,drug
0,TCGA-G2-A2EF,Methotrexate,Partial Response,1,METHOTREXATE
1,TCGA-G2-A2EF,Vinblastine,Partial Response,1,VINBLASTINE
2,TCGA-G2-A2EF,Doxorubicin,Partial Response,1,DOXORUBICIN
3,TCGA-G2-A2EF,Cisplatin,Partial Response,1,CISPLATIN
4,TCGA-DK-A3IQ,Gemcitabine,Stable Disease,0,GEMCITABINE
...,...,...,...,...,...
271,TCGA-QS-A8F1,Carboplatin,Clinical Progressive Disease,0,CARBOPLATIN
272,TCGA-QS-A8F1,Paclitaxel,Clinical Progressive Disease,0,PACLITAXEL
273,TCGA-BG-A0VZ,Cisplatin,Complete Response,1,CISPLATIN
274,TCGA-BG-A0VZ,Paclitaxel,Complete Response,1,PACLITAXEL


In [16]:
predictions = []
y_true = []
drugs = []
for idx, row in test_tcga_with_response.iterrows():
    if row["drug"] in ["CISPLATIN","PACLITAXEL","5-FLUOROURACIL","DOCETAXEL","GEMCITABINE","DOXORUBICIN", "CYCLOPHOSPHAMIDE"]:
        m = mlp(7776, 1, 20)
        m.load_state_dict(
            torch.load(
                f"{MODEL_STATE_PATH}/{row['drug'].replace('/', '_')}.pth",
                map_location=torch.device("cpu"),
            )
        )
        m = m.to(torch.device("cuda:0"))
        inp = torch.from_numpy(tcga_test_df.loc[row["patient.arr"]].values.astype(np.float32)).view(1, -1).to(torch.device("cuda:0"))
        # print(inp.shape)
        pred_train_tcga_y, _ = m(inp)
        predictions.append(nn.Sigmoid()(pred_train_tcga_y).cpu().detach().numpy()[0][0])
        y_true.append(row["response_cat"])
        drugs.append(row["drug"])

In [17]:
len(predictions), len(y_true)

(159, 159)

In [18]:
from sklearn.metrics import roc_auc_score, average_precision_score
auroc = roc_auc_score(y_true, predictions)
auprc = average_precision_score(y_true, predictions)

In [19]:
print(f"AUROC = {auroc}, AUPRC = {auprc}")

AUROC = 0.5110671936758894, AUPRC = 0.7272836197415301


In [20]:
# per drug
res_df = pd.DataFrame(predictions, columns=["y_pred"])
res_df["y_true"] = y_true
res_df["drug"] = drugs

In [21]:
res_df.y_true.value_counts()

y_true
1    115
0     44
Name: count, dtype: int64

In [22]:
for d in res_df.drug.unique():
    try:
        subset_df = res_df[res_df.drug == d]
        auroc = roc_auc_score(subset_df["y_true"], subset_df["y_pred"])
        auprc = average_precision_score(subset_df["y_true"], subset_df["y_pred"])
        print(f"Drug {d} | AUROC = {auroc}, AUPRC = {auprc}")
        print(subset_df.shape)
        print(subset_df.y_true.value_counts())
    except:
        continue

Drug DOXORUBICIN | AUROC = 0.5454545454545454, AUPRC = 0.8757020757020757
(14, 3)
y_true
1    11
0     3
Name: count, dtype: int64
Drug CISPLATIN | AUROC = 0.4965517241379311, AUPRC = 0.7615424846180837
(39, 3)
y_true
1    29
0    10
Name: count, dtype: int64
Drug GEMCITABINE | AUROC = 0.4722222222222222, AUPRC = 0.5166666666666666
(12, 3)
y_true
0    6
1    6
Name: count, dtype: int64
Drug PACLITAXEL | AUROC = 0.56, AUPRC = 0.7803440620788993
(34, 3)
y_true
1    25
0     9
Name: count, dtype: int64
Drug 5-FLUOROURACIL | AUROC = 0.41111111111111115, AUPRC = 0.6905492713413786
(29, 3)
y_true
1    20
0     9
Name: count, dtype: int64
Drug CYCLOPHOSPHAMIDE | AUROC = 0.34375, AUPRC = 0.8895747838762544
(18, 3)
y_true
1    16
0     2
Name: count, dtype: int64
Drug DOCETAXEL | AUROC = 0.75, AUPRC = 0.8334686147186147
(13, 3)
y_true
1    8
0    5
Name: count, dtype: int64
